In [1]:
import os

In [2]:
%pwd

'c:\\Aditya\\Computer Science\\Machine Learning\\projects\\wine-quality-mlops\\research'

In [3]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from src.project.constants import *
from src.project.utils.common import read_yaml, create_directories, save_json


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        return create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/aditya0589/wine-quality-mlops.mlflow"

            
        )
        return model_evaluation_config

In [8]:
import os
import joblib
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from urllib.parse import urlparse
import sys
import mlflow
import mlflow.sklearn
import numpy as np

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        x_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(x_test)
            (rmse, mae, r2) = self.eval_metrics(y_test, predicted_qualities)

            #saving the metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({
                    "rmse": rmse,
                    "mae": mae,
                    "r2": r2
            })

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

        

In [13]:
#pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-09-15 21:04:03,282: INFO: common: created directory at: artifacts]
[2025-09-15 21:04:03,284: INFO: common: created directory at: artifacts/model_evaluation]
[2025-09-15 21:04:22,504: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /aditya0589/wine-quality-mlops.mlflow/api/2.0/mlflow/runs/create]
[2025-09-15 21:04:23,662: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/09/15 21:04:32 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 2
Created version '2' of model 'ElasticnetModel'.
